In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model


In [13]:
df_user = pd.read_csv('../../dataset/user_rating.csv')
place_num = len(df_user.Place_Id.unique())
n_users = len(df_user.User_Id.unique())



In [14]:
df_destination = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';', header=0)
df_destination['price'].fillna(0, inplace=True)
df_destination['category'].fillna('unknown', inplace=True)

In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_user, test_size=0.2, random_state=42)

In [16]:
def create_model():
    # creating destination embedding path
    destination_input = Input(shape=[1])
    destination_embedding = Embedding(place_num+1, 5)(destination_input)
    destination_flat = Flatten()(destination_embedding)

    # creating user embedding path
    user_input = Input(shape=[1])
    user_embedding = Embedding(n_users+1, 5)(user_input)
    user_flat = Flatten()(user_embedding)

    # concatenate features
    conc = Concatenate()([destination_flat, user_flat])

    # add fully-connected-layers
    x = Dense(128, activation='relu')(conc)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)

    # Create model and compile it
    model = Model([user_input, destination_input], output)
    model.compile('adam', 'mean_squared_error')
    history = model.fit([train.User_Id, train.Place_Id], train.Place_Ratings, epochs=20)
    return history, model

Predict

In [23]:
bali_dataset = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';')
bali_dataset
# recommended_places = bali_dataset.loc[recommended_tourism_ids, 'place']
# recommended_places

,index,place,url,is_accessibility,rating,n_reviews,address,price,idx_category,category,description,lat,long,Coordinate
0,0,Pura Puseh Desa Adat Kutri,https://www.google.com/maps/place/Pura+Puseh+D...,0,3.0,1,"F863+W53, Buruan, Kec. Blahbatuh, Kabupaten Gi...",15000,0,Sejarah,Pura Puseh is one of the oldest temples in Bal...,-8.514987,115.298791,"-8.514986706754593, 115.29879103124946"
1,1,Pura Dalem Begawan,https://www.google.com/maps/place/Pura+Dalem+B...,0,3.0,2,"8682+87P, Dauh Puri Kauh, Kec. Denpasar Bar., ...",15000,0,Sejarah,"Located near the Petanu River, Pura Dalem Bega...",-8.682596,115.200610,"-8.682595632877103, 115.20061015733499"
2,2,D'tukad Desa Budaya Kertalangu,https://www.google.com/maps/place/D'tukad+Desa...,0,3.6,9,"9744+WJQ, Kesiman Kertalangu, Kec. Denpasar Ti...",20000,0,Sejarah,D'tukad Desa Budaya Kertalangu is a cultural t...,-8.639347,115.257474,"-8.639347199121982, 115.25747408289917"
3,3,Desa Adat Cangkup,https://www.google.com/maps/place/Desa+Adat+Ca...,0,3.7,3,"Pupuan Sawah, Kec. Selemadeg, Kabupaten Tabana...",15000,0,Sejarah,Desa Adat Cangkup is a traditional village tha...,-8.439189,115.051934,"-8.43918883821229, 115.05193408611053"
4,4,Monumen Perjuangan Desa Tegaljadi,https://www.google.com/maps/place/Monumen+Perj...,0,4.0,1,"G585+J3, Tegaljadi, Kec. Marga, Kabupaten Taba...",15000,0,Sejarah,This monument was erected to commemorate the s...,-8.483242,115.157706,"-8.4832418767373, 115.15770565767143"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,Banjar Hot Spring,https://www.google.com/maps/place/Banjar+Hot+S...,0,4.3,2422,"Jalan Banjar, Banjar, Kec. Banjar, Kabupaten B...",20000,1,Wisata Alam,Hot springs complex surrounded by tropical gar...,-8.210255,114.967123,"-8.210254599982365, 114.96712273733684"
191,191,Bali Pulina,https://www.google.com/maps/place/Bali+Pulina/...,0,4.5,3960,"Jl. Raya Pujung Kaja No.Br, Sebatu, Kec. Tegal...",100000,1,Wisata Alam,"Sprawling venue with lush, tropical vegetation...",-8.422674,115.278990,"-8.422673949154303, 115.27899012017349"
192,192,Goa Lawah Temple,https://www.google.com/maps/place/Goa+Lawah+Te...,0,4.7,4479,"CFX9+9GG, Jl. Raya Goa Lawah, Pesinggahan, Kec...",20000,0,Sejarah,"This small, 11th-century Balinese Hindu temple...",-8.551260,115.468823,"-8.551260117939812, 115.46882253550854"
193,193,Pantai Batu Bolong,https://www.google.com/maps/place/Pantai+Batu+...,0,4.4,7248,"Jl. Pantai Batu Bolong, Canggu, Kec. Kuta Utar...",15000,2,Pantai,This well-known sandy beach near shops & resta...,-8.658103,115.130329,"-8.658103348069142, 115.13032921850571"


In [18]:
history, model = create_model()
model.save('../model.h5')


Epoch 1/20

229/229 [==============================] - 3s 4ms/step - loss: 3.0470
Epoch 2/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8728
Epoch 3/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8382
Epoch 4/20
229/229 [==============================] - 1s 3ms/step - loss: 1.8184
Epoch 5/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8099
Epoch 6/20
229/229 [==============================] - 1s 3ms/step - loss: 1.8038
Epoch 7/20
229/229 [==============================] - 1s 4ms/step - loss: 1.7992
Epoch 8/20
229/229 [==============================] - 1s 3ms/step - loss: 1.7967
Epoch 9/20
229/229 [==============================] - 1s 4ms/step - loss: 1.7915
Epoch 10/20
229/229 [==============================] - 1s 3ms/step - loss: 1.7708
Epoch 11/20
229/229 [==============================] - 1s 3ms/step - loss: 1.7308
Epoch 12/20
229/229 [==============================] - 1s 3ms/step - loss: 1.6802
Epoch 13/20
229/229 [==

c:\Users\u\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
def create_final_data(list_of_idx):
    new_df = df_destination[df_destination.index.isin(list_of_idx)].copy()
    list_of_dest = []

    for row in new_df.iterrows():
        dest_dict = {}
        dest_dict['idx_place'] = row[1]['index']
        dest_dict['place'] = row[1]['place']
        dest_dict['url'] = row[1]['url']
        dest_dict['address'] = row[1]['address']
        dest_dict['is_accessibility'] = row[1]['is_accessibility']
        dest_dict['rating'] = row[1]['rating']
        dest_dict['n_reviews'] = row[1]['n_reviews']
        dest_dict['price'] = row[1]['price']
        dest_dict['category'] = row[1]['category']
        dest_dict['description'] = row[1]['description']
        list_of_dest.append(dest_dict)
    
    return list_of_dest

In [26]:
model.save('../model.h5')
model = tf.keras.models.load_model('../model.h5')


c:\Users\u\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
def might_like(idx_user, model=model):
    tourism_data = np.array(list(set(df_user.Place_Id)))
    user = np.array([idx_user]*len(tourism_data))

    
    predictions = model.predict([user, tourism_data])
    predictions = np.array([a[0] for a in predictions])
    
    recommended_tourism_idx = (-predictions).argsort()[:10]
    
    #convert index to data place
    data = create_final_data(recommended_tourism_idx)
    return data

In [28]:
idx_user = 5
get_recom = might_like(idx_user)
get_recom

7/7 [==============================] - 0s 2ms/step


[{'idx_place': 1,
  'place': 'Pura Dalem Begawan',
  'url': 'https://www.google.com/maps/place/Pura+Dalem+Begawan/@-8.6841653,115.1646471,14z/data=!4m10!1m2!2m1!1smuseum+di+bali!3m6!1s0x2dd240d081b84ebb:0xd9bca659141d6c7c!8m2!3d-8.6841653!4d115.200696!15sCg5tdXNldW0gZGkgYmFsaZIBBm11c2V1beABAA!16s%2Fg%2F11c5s513p3?entry=ttu',
  'address': '8682+87P, Dauh Puri Kauh, Kec. Denpasar Bar., Kota Denpasar, Bali 80221',
  'is_accessibility': 0,
  'rating': 3.0,
  'n_reviews': 2,
  'price': 15000,
  'category': 'Sejarah',
  'description': 'Located near the Petanu River, Pura Dalem Begawan is a sacred site dedicated to Lord Shiva in his manifestation as the ruler of the underworld (Dalem). This temple is known for its beautiful carvings and often hosts major traditional ceremonies.'},
 {'idx_place': 15,
  'place': 'Sidi Tapa Waterfall',
  'url': 'https://www.google.com/maps/place/Sidi+Tapa+Waterfall/@-8.3551145,114.702788,17z/data=!3m1!4b1!4m6!3m5!1s0x2dd1634010b78511:0x6d4495462805230d!8m2!3d-8.